In [ ]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

# Excel dosyasını oku
df = pd.read_excel("/content/covidsonrasi.xlsx")
sentences = df["tweets"].values  # Cümleler
labels = df["total"].values  # Sınıf etiketleri

# BERT tokenizer'ı yükle
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Veri seti sınıfı oluştur
class TweetDataset(Dataset):
    def __init__(self, sentences, labels, tokenizer, max_length):
        self.sentences = sentences
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        sentence = str(self.sentences[idx])
        label = self.labels[idx]

        # Cümleyi BERT için girişe uygun hâle getir
        encoded_inputs = self.tokenizer.encode_plus(
            sentence,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        input_ids = encoded_inputs['input_ids'].squeeze()
        attention_mask = encoded_inputs['attention_mask'].squeeze()

        return input_ids, attention_mask, label

# Veri kümesi ve veri yükleyici oluştur
max_length = 128
dataset = TweetDataset(sentences, labels, tokenizer, max_length)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# BERT modelini yükle
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Aygıtı belirle (GPU kullanılacaksa)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Modeli eğit
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

for epoch in range(2):
    model.train()
    total_loss = 0

    for batch in dataloader:
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)

        optimizer.zero_grad()

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    average_loss = total_loss / len(dataloader)
    print("Epoch {}: Average Loss = {:.4f}".format(epoch+1, average_loss))




ValueError: ignored

In [ ]:
pip install --upgrade transformers



SyntaxError: ignored

In [ ]:
model.eval()  # Modeli değerlendirme moduna geçir

predictions = []
true_labels = []

for batch in dataloader:
    input_ids = batch[0].to(device)
    attention_mask = batch[1].to(device)
    labels = batch[2].to('cpu')  # Etiketleri CPU'ya taşı

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)

    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=1)
    predicted_labels = torch.argmax(probabilities, dim=1)

    predictions.extend(predicted_labels.tolist())
    true_labels.extend(labels.tolist())

# Accuracy
accuracy = accuracy_score(true_labels, predictions)
print("Accuracy: {:.4f}".format(accuracy))

# F-measure, precision ve recall
f_measure = f1_score(true_labels, predictions)
precision = precision_score(true_labels, predictions)
recall = recall_score(true_labels, predictions)

print("F-measure: {:.4f}".format(f_measure))
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))



In [ ]:
# Modelin kaydedileceği dosya yolu
model_save_path = "/content/bert_model.pt"

# Modelin eğitimi tamamlandıktan sonra kaydetme
torch.save(model.state_dict(), model_save_path)

print("Model kaydedildi:", model_save_path)

Model kaydedildi: /content/bert_model.pt


In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Modeli değerlendirme fonksiyonu
def evaluate_model(model, dataloader):
    model.eval()
    predictions = []
    true_labels = []

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch[0].to(device)
            attention_mask = batch[1].to(device)
            labels = batch[2].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            probabilities = torch.softmax(logits, dim=1)
            predicted_labels = torch.argmax(probabilities, dim=1)

            predictions.extend(predicted_labels.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions)
    recall = recall_score(true_labels, predictions)
    f1 = f1_score(true_labels, predictions)

    return accuracy, precision, recall, f1

dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

for epoch in range(1):
    model.train()
    total_loss = 0





    # Değerlendirme işlemi
    accuracy, precision, recall, f1 = evaluate_model(model, dataloader)

    print("Accuracy: {:.2f}".format(accuracy * 100))
    print("Precision: {:.2f}".format(precision * 100))
    print("Recall: {:.2f}".format(recall * 100))
    print("F1-score: {:.2f}".format(f1 * 100))


In [ ]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Excel dosyasını oku
df = pd.read_excel("/content/covidsonrasi.xlsx")
sentences = df["A"].values  # Cümleler
labels = df["B"].values  # Sınıf etiketleri

# BERT tokenizer'ı yükle
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Veri seti sınıfı oluştur
class TweetDataset(Dataset):
    def __init__(self, sentences, labels, tokenizer, max_length):
        self.sentences = sentences
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        sentence = str(self.sentences[idx])
        label = self.labels[idx]

        # Cümleyi BERT için girişe uygun hâle getir
        encoded_inputs = self.tokenizer.encode_plus(
            sentence,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        input_ids = encoded_inputs['input_ids'].squeeze()
        attention_mask = encoded_inputs['attention_mask'].squeeze()

        return input_ids, attention_mask, label

# Veri kümesi ve veri yükleyici oluştur
max_length = 128
dataset = TweetDataset(sentences, labels, tokenizer, max_length)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# BERT modelini yükle
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Aygıtı belirle (GPU kullanılacaksa)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Modeli eğit
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

for epoch in range(2):
    model.train()
    total_loss = 0

    for batch in dataloader:
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)

        optimizer.zero_grad()

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    average_loss = total_loss / len(dataloader)
    print("Epoch {}: Average Loss = {:.4f}".format(epoch+1, average_loss))

# Modeli değerlendir
model.eval()
predictions = []
true_labels = []

with torch.no_grad():
    for batch in dataloader:
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        probabilities = torch.softmax(logits, dim=1)
        predicted_labels = torch.argmax(probabilities, dim=1)

        predictions.extend(predicted_labels.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(true_labels, predictions)
precision = precision_score(true_labels, predictions)
recall = recall_score(true_labels, predictions)
f1 = f1_score(true_labels, predictions)

print("Accuracy: {:.2f}".format(accuracy * 100))
print("Precision: {:.2f}".format(precision * 100))
print("Recall: {:.2f}".format(recall * 100))
print("F1-score: {:.2f}".format(f1 * 100))


ValueError: ignored

In [ ]:
pip install pandas torch transformers scikit-learn


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas
import torch
import transformers
import sklearn

print("pandas version:", pandas.__version__)
print("torch version:", torch.__version__)
print("transformers version:", transformers.__version__)
print("scikit-learn version:", sklearn.__version__)


ValueError: ignored

In [ ]:
pip install pandas torch transformers scikit-learn


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
